In [1]:
{-# LANGUAGE NegativeLiterals #-}
{-# LANGUAGE FlexibleContexts #-}
{-# LANGUAGE OverloadedStrings #-}
{-# LANGUAGE TypeFamilies #-}
{-# LANGUAGE BangPatterns #-}

In [2]:
import Data.Text (Text)
import qualified Data.Text as T
import qualified Data.Text.IO as TIO

import Text.Megaparsec hiding (State)
import qualified Text.Megaparsec.Lexer as L
import Text.Megaparsec.Text (Parser)
import qualified Control.Applicative as CA
-- import Data.Functor (void)

import qualified Data.Map.Strict as M
import Data.Map.Strict ((!))

-- import Data.Vector ((!), (//))
-- import qualified Data.Vector as V

import Data.List 
import qualified Data.Functor as F

In [3]:
type Grid = M.Map (Int, Int) Bool
-- type Grid = [[Char]]
-- type Grid = [[Bool]]

type ExplodedGrid = M.Map (Int, Int) Grid

data Rule = Rule Grid Grid deriving (Eq, Show)

rulePre (Rule g _) = g
rulePost (Rule _ g) = g

In [38]:
onlySpace = (char ' ') <|> (char '\t')

sc :: Parser ()
sc = L.space (skipSome onlySpace) CA.empty CA.empty

In [5]:
lexeme  = L.lexeme sc

symbol = L.symbol sc
rowSep = symbol "/"
ruleJoin = symbol "=>"

-- present :: Parser Bool
present = id True <$ symbol "#"

-- absent :: Parser Bool
absent = id False <$ symbol "."

rulesP = ruleP `sepBy` space
ruleP = Rule <$> gridP <*> (ruleJoin *> gridP)

gridP = gridify <$> rowP `sepBy` rowSep
    where gridify g = M.fromList $ concat 
                                    [map (\(c, v) -> ((r, c), v)) nr | 
                                             (r, nr) <- zip [0..] 
                                                            [zip [0..] r | r <- g]]


rowP = some (present <|> absent)
 
successfulParse :: Text -> [Rule]
successfulParse input = 
        case parse rulesP "input" input of
                Left  _error -> [] -- TIO.putStr $ T.pack $ parseErrorPretty err
                Right instructions  -> instructions

In [6]:
parseTest ruleP "#./.. => .##/.##/#.."

Rule (fromList [((0,0),True),((0,1),False),((1,0),False),((1,1),False)]) (fromList [((0,0),False),((0,1),True),((0,2),True),((1,0),False),((1,1),True),((1,2),True),((2,0),True),((2,1),False),((2,2),False)])

In [7]:
parseTest ruleP "##./#.#/... => #.#./.#../.##./...#"

Rule (fromList [((0,0),True),((0,1),True),((0,2),False),((1,0),True),((1,1),False),((1,2),True),((2,0),False),((2,1),False),((2,2),False)]) (fromList [((0,0),True),((0,1),False),((0,2),True),((0,3),False),((1,0),False),((1,1),True),((1,2),False),((1,3),False),((2,0),False),((2,1),True),((2,2),True),((2,3),False),((3,0),False),((3,1),False),((3,2),False),((3,3),True)])

In [8]:
testRule = head $ successfulParse  "##./#.#/... => #.#./.#../.##./...#"

In [9]:
testRule

Rule (fromList [((0,0),True),((0,1),True),((0,2),False),((1,0),True),((1,1),False),((1,2),True),((2,0),False),((2,1),False),((2,2),False)]) (fromList [((0,0),True),((0,1),False),((0,2),True),((0,3),False),((1,0),False),((1,1),True),((1,2),False),((1,3),False),((2,0),False),((2,1),True),((2,2),True),((2,3),False),((3,0),False),((3,1),False),((3,2),False),((3,3),True)])

In [10]:
parseTest rulesP "#./.. => .##/.##/#..\n##./#.#/... => #.#./.#../.##./...#"

[Rule (fromList [((0,0),True),((0,1),False),((1,0),False),((1,1),False)]) (fromList [((0,0),False),((0,1),True),((0,2),True),((1,0),False),((1,1),True),((1,2),True),((2,0),True),((2,1),False),((2,2),False)]),Rule (fromList [((0,0),True),((0,1),True),((0,2),False),((1,0),True),((1,1),False),((1,2),True),((2,0),False),((2,1),False),((2,2),False)]) (fromList [((0,0),True),((0,1),False),((0,2),True),((0,3),False),((1,0),False),((1,1),True),((1,2),False),((1,3),False),((2,0),False),((2,1),True),((2,2),True),((2,3),False),((3,0),False),((3,1),False),((3,2),False),((3,3),True)])]

Rule (fromList [((0,0),True),((0,1),False),((1,0),False),((1,1),False)]) 
     (fromList [((0,0),False),((0,1),True),((0,2),True),((1,0),False),((1,1),True),((1,2),True),((2,0),True),((2,1),False),((2,2),False),((2,3),True),((2,4),True),((2,5),False),((3,0),True),((3,1),False),((3,2),True),((4,0),False),((4,1),False),((4,2),False)]),
     
Rule (fromList []) 
     (fromList [((0,0),True),((0,1),False),((0,2),True),((0,3),False),((1,0),False),((1,1),True),((1,2),False),((1,3),False),((2,0),False),((2,1),True),((2,2),True),((2,3),False),((3,0),False),((3,1),False),((3,2),False),((3,3),True)])

In [11]:
g = [[False,True,True],[False,True,True],[True,False,False]]

In [12]:
concat [map (\(c, v) -> ((r, c), v)) nr | (r, nr) <- zip [0..] [zip [0..] r | r <- g]]

[((0,0),False),((0,1),True),((0,2),True),((1,0),False),((1,1),True),((1,2),True),((2,0),True),((2,1),False),((2,2),False)]

In [53]:
bounds :: M.Map (Int, Int) a -> (Int, Int)
bounds grid = (maximum $ map fst $ M.keys grid, maximum $ map snd $ M.keys grid)

In [54]:
bounds (rulePost testRule)

(3,3)

In [15]:
showGrid g = unlines [[showChar $ M.findWithDefault False (r, c) g | 
                c <- [0..cm] ] | r <- [0..rm] ]
    where (rm, cm) = bounds g
          showChar True = '#'
          showChar False = '.'

In [16]:
putStrLn $ showGrid $ rulePost testRule

#.#.
.#..
.##.
...#

In [17]:
initialGrid = case parse gridP "" ".#./..#/###" of 
                Left _ -> M.empty 
                Right g -> g

In [18]:
initialGrid

fromList [((0,0),False),((0,1),True),((0,2),False),((1,0),False),((1,1),False),((1,2),True),((2,0),True),((2,1),True),((2,2),True)]

In [19]:
putStrLn $ showGrid initialGrid

.#.
..#
###

In [20]:
reflectH g = M.fromList [((r, c) , M.findWithDefault False (rm - r, c) g) | r <- [0..rm], c <- [0..cm] ]
    where (rm, cm) = bounds g

In [21]:
putStrLn $ showGrid $ reflectH initialGrid

###
..#
.#.

In [22]:
reflectV g = M.fromList [((r, c) , M.findWithDefault False (r, cm - c) g) | r <- [0..rm], c <- [0..cm] ]
    where (rm, cm) = bounds g

In [23]:
putStrLn $ showGrid $ reflectV initialGrid

.#.
#..
###

In [24]:
transpose g = M.fromList [((c, r) , M.findWithDefault False (r, c) g) | r <- [0..rm], c <- [0..cm] ]
    where (rm, cm) = bounds g

In [25]:
putStrLn $ showGrid $ transpose initialGrid

..#
#.#
.##

In [26]:
allArrangements grid = map (\f -> f grid) [ id
                                          , reflectH
                                          , reflectV
                                          , transpose
                                          , reflectH . transpose
                                          , reflectV . transpose
                                          , reflectH . reflectV . transpose
                                          , reflectV . reflectH
                                          ]

In [27]:
map showGrid $ allArrangements initialGrid

[".#.\n..#\n###\n","###\n..#\n.#.\n",".#.\n#..\n###\n","..#\n#.#\n.##\n",".##\n#.#\n..#\n","#..\n#.#\n##.\n","##.\n#.#\n#..\n","###\n#..\n.#.\n"]

In [28]:
sampleRulesCompact = successfulParse "../.# => ##./#../...\n.#./..#/### => #..#/..../..../#..#"
length sampleRulesCompact

2

In [29]:
expandRule rule = [Rule l (rulePost rule) | l <- allArrangements (rulePre rule)]
expandRules = concatMap expandRule

In [30]:
[showGrid (rulePre r) ++ "=>" ++ showGrid (rulePost r) | r <- expandRule testRule]

["##.\n#.#\n...\n=>#.#.\n.#..\n.##.\n...#\n","...\n#.#\n##.\n=>#.#.\n.#..\n.##.\n...#\n",".##\n#.#\n...\n=>#.#.\n.#..\n.##.\n...#\n","##.\n#..\n.#.\n=>#.#.\n.#..\n.##.\n...#\n",".#.\n#..\n##.\n=>#.#.\n.#..\n.##.\n...#\n",".##\n..#\n.#.\n=>#.#.\n.#..\n.##.\n...#\n",".#.\n..#\n.##\n=>#.#.\n.#..\n.##.\n...#\n","...\n#.#\n.##\n=>#.#.\n.#..\n.##.\n...#\n"]

In [31]:
length $ expandRules sampleRulesCompact

16

In [32]:
readRules = expandRules . successfulParse

In [33]:
sampleRules = readRules "../.# => ##./#../...\n.#./..#/### => #..#/..../..../#..#"
sampleRules

[Rule (fromList [((0,0),False),((0,1),False),((1,0),False),((1,1),True)]) (fromList [((0,0),True),((0,1),True),((0,2),False),((1,0),True),((1,1),False),((1,2),False),((2,0),False),((2,1),False),((2,2),False)]),Rule (fromList [((0,0),False),((0,1),True),((1,0),False),((1,1),False)]) (fromList [((0,0),True),((0,1),True),((0,2),False),((1,0),True),((1,1),False),((1,2),False),((2,0),False),((2,1),False),((2,2),False)]),Rule (fromList [((0,0),False),((0,1),False),((1,0),True),((1,1),False)]) (fromList [((0,0),True),((0,1),True),((0,2),False),((1,0),True),((1,1),False),((1,2),False),((2,0),False),((2,1),False),((2,2),False)]),Rule (fromList [((0,0),False),((0,1),False),((1,0),False),((1,1),True)]) (fromList [((0,0),True),((0,1),True),((0,2),False),((1,0),True),((1,1),False),((1,2),False),((2,0),False),((2,1),False),((2,2),False)]),Rule (fromList [((0,0),False),((0,1),True),((1,0),False),((1,1),False)]) (fromList [((0,0),True),((0,1),True),((0,2),False),((1,0),True),((1,1),False),((1,2),False

In [125]:
text <- TIO.readFile "../../data/advent21.txt"
rules = readRules text

In [126]:
length rules

864

In [34]:
apply rules grid = rulePost thisRule
    where ri = head $ findIndices (\r -> rulePre r == grid) rules
          thisRule = rules!!ri

In [37]:
putStrLn $ showGrid $ apply sampleRules initialGrid

#..#
....
....
#..#

In [49]:
subGrid :: Int -> Grid -> Int -> Int -> Grid
subGrid n g bigR bigC = M.fromList [ ((r, c), 
                                      M.findWithDefault False (r + rStep, c + cStep) g) 
                                   | r <- [0..(n - 1)], c <- [0..(n - 1)]
                                   ]
    where rStep = bigR * n
          cStep = bigC * n

In [50]:
explodeGrid' :: Int -> Grid -> ExplodedGrid
explodeGrid' n g = M.fromList [((bigR, bigC), subGrid n g bigR bigC) | bigR <- [0..bigRm], bigC <- [0..bigCm]]
    where (rm, cm) = bounds g
          bigRm = (rm + 1) `div` n - 1
          bigCm = (cm + 1) `div` n - 1

In [107]:
explodeGrid :: Grid -> ExplodedGrid
explodeGrid g = if (rm + 1) `rem` 2 == 0 
                then explodeGrid' 2 g
                else explodeGrid' 3 g
    where (rm, cm) = bounds g

In [108]:
testEg = explodeGrid $ apply sampleRules initialGrid
M.map showGrid testEg

fromList [((0,0),"#.\n..\n"),((0,1),".#\n..\n"),((1,0),"..\n#.\n"),((1,1),"..\n.#\n")]

In [77]:
explodedRows eg = [M.filterWithKey (\(r, _) _ -> r == row) eg | row <- [0..rowMax] ]
    where (rowMax, _) = bounds eg

In [78]:
explodedRows testEg

[fromList [((0,0),fromList [((0,0),True),((0,1),False),((1,0),False),((1,1),False)]),((0,1),fromList [((0,0),False),((0,1),True),((1,0),False),((1,1),False)])],fromList [((1,0),fromList [((0,0),False),((0,1),False),((1,0),True),((1,1),False)]),((1,1),fromList [((0,0),False),((0,1),False),((1,0),False),((1,1),True)])]]

In [79]:
(>-<) g1 g2 = M.union g1 g2'
    where (_, cm) = bounds g1
          g2' = M.mapKeys (\(r, c) -> (r, c + cm + 1)) g2

In [80]:
(>|<) g1 g2 = M.union g1 g2'
    where (rm, _) = bounds g1
          g2' = M.mapKeys (\(r, c) -> (r + rm + 1, c)) g2

In [81]:
M.findWithDefault M.empty (0, 0) testEg

fromList [((0,0),True),((0,1),False),((1,0),False),((1,1),False)]

In [82]:
showGrid $ (M.findWithDefault M.empty (0, 1) testEg) >-< (M.findWithDefault M.empty (0, 0) testEg)

Line 1: Redundant bracket
Found:
(M.findWithDefault M.empty (0, 1) testEg) >-<
  (M.findWithDefault M.empty (0, 0) testEg)
Why not:
M.findWithDefault M.empty (0, 1) testEg >-<
  (M.findWithDefault M.empty (0, 0) testEg)Line 1: Redundant bracket
Found:
(M.findWithDefault M.empty (0, 1) testEg) >-<
  (M.findWithDefault M.empty (0, 0) testEg)
Why not:
(M.findWithDefault M.empty (0, 1) testEg) >-<
  M.findWithDefault M.empty (0, 0) testEg

".##.\n....\n"

In [83]:
showGrid $ (M.findWithDefault M.empty (0, 1) testEg) >|< (M.findWithDefault M.empty (0, 0) testEg)

Line 1: Redundant bracket
Found:
(M.findWithDefault M.empty (0, 1) testEg) >|<
  (M.findWithDefault M.empty (0, 0) testEg)
Why not:
M.findWithDefault M.empty (0, 1) testEg >|<
  (M.findWithDefault M.empty (0, 0) testEg)Line 1: Redundant bracket
Found:
(M.findWithDefault M.empty (0, 1) testEg) >|<
  (M.findWithDefault M.empty (0, 0) testEg)
Why not:
(M.findWithDefault M.empty (0, 1) testEg) >|<
  M.findWithDefault M.empty (0, 0) testEg

".#\n..\n#.\n..\n"

In [88]:
contractExploded :: ExplodedGrid -> Grid
contractExploded gs = foldl1 (>|<) $ map (foldl1 (>-<)) rows
    where rows = explodedRows gs

In [90]:
showGrid $ contractExploded testEg

"#..#\n....\n....\n#..#\n"

In [94]:
putStrLn $ showGrid $ contractExploded $ M.map (apply sampleRules) testEg

##.##.
#..#..
......
##.##.
#..#..
......

In [128]:
applyOnce rules g = contractExploded $ M.map (apply rules) $ explodeGrid g

In [135]:
putStrLn $ showGrid $ (!! 5) $ iterate (applyOnce rules) initialGrid

.#.#.#.#..##..#.##
.#.#...#..##....##
..####..##..####..
..#.#..##.###.#.#.
....#..##.###...#.
###..##..#..###..#
..#.##..#.##.##...
....##....##.##.##
####..####..#..##.
#.#.#.#.#.#.....#.
#...#.#...#..##.#.
###..####..###...#
..#.##..#.##..#.##
....##....##....##
####..####..####..
#.#.#.#.#.#.#.#.#.
#...#.#...#.#...#.
###..####..####..#

In [131]:
putStrLn $ showGrid $ (!(1, 2)) $ explodeGrid $ last $ take 3 $ iterate (applyOnce rules) initialGrid

.#
##

In [136]:
countLit = M.size . M.filter id

In [137]:
countLit initialGrid

5

In [142]:
putStrLn $ showGrid $ (!! 2) $ iterate (applyOnce sampleRules) initialGrid

##.##.
#..#..
......
##.##.
#..#..
......

In [143]:
countLit $ (!! 2) $ iterate (applyOnce sampleRules) initialGrid

12

In [144]:
countLit $ (!! 5) $ iterate (applyOnce rules) initialGrid

158

In [145]:
countLit $ (!! 18) $ iterate (applyOnce rules) initialGrid

2301762